## M20732/M342 
## Statistics Methods in Health Research & Social Science  
### Coursework (2022-23)  
---
### <font color=RoyalBlue> UP960303 </font>

### Instructions 
1. Your solutions must be submitted on Moodle no later than 23:55 pm on Friday 30 November 2022.
2. This coursework is divided into two parts, each worth 20% of the total mark for
this module. Answer all questions.
3. You must write up your work individually.
4. You must use R-Colab (or R studio) for one of the two parts.  
**<font color=RoyalBlue>Please write comments in your code and explain clearly what you are doing and the theoretical method to find the
solutions.</font>**
5. You need to show all steps in your calculations.
6. To get a good mark you must **<font color=RoyalBlue>justify every claim</font>**.
---

My comments will be denoted using comments within the code and with an overall markdown explanations in **<font color=RoyalBlue> blue</font>**.  
My findings from the statistical analysis will be in **<font color=Green> green </font>**.

This is where I have installed all of the libraries that I will need for the coursework so that they can run before the rest of the code.

In [ ]:
# install.packages("readxl")
# install.packages("gmodels")
# install.packages("samplesizeCMH")
# install.packages("exact2x2")
# install.packages("ResourceSelection")
# install.packages("lmtest")
# install.packages("mosaic")
# install.packages("latticeExtra")
# install.packages("survival")
# install.packages("survminer")
# install.packages("doBy")


In [ ]:
library(tidyverse) 
library(haven)
library(epitools)
library(readxl)
library(gmodels)
library(samplesizeCMH)
library(exact2x2)
library(ResourceSelection)
library(lmtest)
library(mosaic)
library(latticeExtra)
library(survival)
library(survminer)
library(doBy)

## Part 2
### These are the exercises that must be solved with R.
---

### Question D
A (fictitious) study was conducted to investigate the effect of cervical length, maternal characteristics and induced labour on the risk of a C-section delivery. Data were collected for 1028
women. Age, ethnicity, BMI and the number of previous pregnancies (parity) were among the
data collected. The Excel data file csw 2022 LogData.xlsx supplied for this assignment contains
the following information relating to:

In [ ]:
# Here I will use the methods from previous lectures to abstract the columns into a table 
QuestionDData <- read_excel("CSW_2022_LogData.xlsx")
# I also need to rename columns in order to use then effectively within R 
colnames(QuestionDData)=c("PatientID","MA", "Ethnicity", "BMI","BMIgroup", "NoPreviousPregnancys", "FirstBaby", "CxLength", "CxGroup","Spontaneous","Caesarean" )
QuestionDData$Delivery <- QuestionDData$Caesarean
# Change the Data in Caesarean so that its a 1 or a 0 using the pipe operator
QuestionDData <- QuestionDData %>%
      mutate(Caesarean = ifelse(Caesarean == 1 ,0,1))
# Here im changing the data for First baby from 1/2 to yes/no
QuestionDData <- QuestionDData %>%
      mutate(FirstBaby = ifelse(FirstBaby == 1 ,'Yes', "No"))
# Here I am showing the top of the table 
head(QuestionDData)
# This table call is commented out but when in play shows the full table of data 
# QuestionDData

1. [3 marks] Ignoring all other variables, determine whether there is evidence of an association between Caesarean delivery and BMI group. Interpret your finding.

<font color=RoyalBlue> Here I am creating a table for the results being tested. Then I run a Chi-Square test to determine whether there is an assosiation between having a Caesarean and BMI group </font>

In [ ]:
TblCeaVSBMIG = table(QuestionDData$BMIgroup,QuestionDData$Caesarean)
colnames(TblCeaVSBMIG) <- c( "Vaginal", "Caesarean")
addmargins(TblCeaVSBMIG)

chisq.test(TblCeaVSBMIG ,correct = FALSE)


<font color=Green> The Chi Square statistic is 19.787, the P value is 0.000051. The p-value is less than 0.05 hence the result is significant. </font>

2. [3 marks] Obtain the odds ratios (and their CI) to describe the effect of BMI group on the
incidence of Caesarean delivery in this study population and interpret your result. Use the
first tertile (=1) as comparator group.

<font color=RoyalBlue> Code explanation here... </font>

In [ ]:
oddsratio(TblCeaVSBMIG,correction=FALSE,verbose=TRUE,conf.level=0.05,method="wald")

3. [10 marks] Carry out a stratified analysis, using Firstbaby as first variable and Cx-group as the stratification variable.   
- Is there any evidence that Firstbaby has an effect on Delivery?   
- Is there any evidence that the risk due to Firstbaby is modified by the cervix size?   
- What are the odds ratio for Firstbaby (yes) after adjusting for Cx-group?

<font color=RoyalBlue> I need to extract the data for:   
- First Baby  
- Cx-Group  
- Delivery  
- Cervix Size  
 </font>

In [ ]:
# First we need to seperate data into the different Cx Groups 
# Ill start by extracting  the data we need using aggregate
QDThreeData = aggregate(Caesarean ~ CxGroup + Delivery + FirstBaby , data = QuestionDData, FUN=length)
View(QDThreeData)

In [ ]:
# Now I am creating a table out of the results that we need to evaluate
QDThreeTable = xtabs(Caesarean ~  FirstBaby + Delivery + CxGroup, data=QDThreeData)
ftable(QDThreeTable)

In [ ]:
# Now I'm finishing the tables by adding margins and spliting them by CxGroup

addmargins(QDThreeTable, margin = seq_along(dim(QDThreeTable)), FUN = sum, quiet = FALSE)

In [ ]:
# Now we will analyse the data by peforming a CHI-Squared test 
apply(QDThreeTable, 3, chisq.test,correct = FALSE)

Is there any evidence that Firstbaby has an effect on Delivery?
<font color=Green> 

I'm not super sure 

</font>

In [ ]:
# Creating a table in order to be able to compare results that haven't been adjusted
QDThreeTableCompare = table(QuestionDData$FirstBaby, QuestionDData$Delivery ) 
QDThreeTableCompare

# Unadjusted Chi-Squared Test
CrossTable(QuestionDData$FirstBaby, QuestionDData$Delivery, expected = TRUE)


Is there any evidence that the risk due to Firstbaby is modified by the cervix size?

<font color=Green> 

I'm not super sure 

</font>

<font color=RoyalBlue> In this code block I am running Odds Ratio on the 3 different CX-Groups and then I create a data table to run a odds ration for the total OR , then I run a odds ratio test without adjusting for CX-group. </font>

In [ ]:
# Odds ratio for FirstBaby resulting in a casearean after adjusting for Cx-group
apply(QDThreeTable, 3, oddsratio.wald)
# Odds ratio for FirstBaby resulting in a Caesarean without adjusting for CX-group
oddsratio(QDThreeTableCompare,method="wald",conf.level=0.95)

What are the odds ratio for Firstbaby (yes) after adjusting for Cx-group?
<font color=Green> 

This table shows all the results I've found  

|CX-Group | Risk Estimate| Value	| 95% CI |
|-------|--------------|--------|---------------------|
|1 |	OR for First Baby (yes/no)|	9.551 |	(2.223,	41.034) |
|2	| OR for First Baby (yes/no)|	4.635 |	(1.895,	11.334) |
|3	| OR for First Baby (yes/no)|	4.280  |	(2.357,	7.774)    |
|Total|	OR for First Baby (yes/no)|	4.645  |	(2.952,	7.311)   | 
  
From these numbers we can see that:   
- Members of CX-Group 1 are 9.6 times more likely to have a casesarean when its their first baby.
- Members of CX-Group 2 are 4.6 times more likely to have a casesarean when its their first baby.
- Members of CX-Group 3 are 4.3 times more likely to have a casesarean when its their first baby.

From these results we see that CX-group 1 appears to be more likely(in terms of odds) for having a Casesarean when its thier first birth than the other CX-Groups

</font>

4. [5 marks] Perform a multivariate analysis (logistic regression) to explore the relationship
between delivery and all seven of the potential predictor variables recorded (Do not use
group variables). (Hint: Your outcome of interest is Caesarean section yes or no, code the
outcome using a dummy variable in a suitable way).

<font color=RoyalBlue> Code explanation here... </font>

In [ ]:
multivariateAnalysis = glm(Caesarean ~ Ethnicity + MA + BMI + NoPreviousPregnancys + FirstBaby + CxLength + Spontaneous, data=QuestionDData, family = binomial(link="logit"))

summary(multivariateAnalysis)

5. [3 marks] Find out and comment on the goodness of fit of your model.

<font color=RoyalBlue> Code explanation here... </font>

In [ ]:
# Not sure about this bit :/
# myplot = xyplot(jitter(Disease) ~ Age, groups=Gender, data=MyData, alpha=0.5, pch=19, cex=2, ylab="Disease") 
# fit.outcome = makeFun(logm) 
# plotFun(fit.outcome(Age=x, Gender=1) ~ x, lwd=3, plot=myplot, add=TRUE)

6. [10 marks] Describe the effect of each predictor on the likelihood of caesarean delivery.

<font color=RoyalBlue> Code explanation here... </font>

---
### Question E
Using the data from the randomised control trial comparing treatments for ulcerative colitis  
(Question C):  

| Randomised Control Trial |   |   |   |    |   |    |    |     |     |    |     |    |     |     |    |     |     |     |     |     |     |     |     |
|--------------------------|---|---|---|----|---|----|----|-----|-----|----|-----|----|-----|-----|----|-----|-----|-----|-----|-----|-----|-----|-----|
| Test group:              | 2 | 6 | 6 | 6+ | 7 | 9+ | 10 | 10+ | 11+ | 13 | 17+ | 18 | 19+ | 20+ | 22 | 24+ | 25+ | 32+ | 32+ | 34+ | 35+ | 36+ |     |
| Control group:           | 1 | 2 | 2 | 3  | 4 | 4  | 5  | 5   | 8   | 8  | 10  | 11 | 11+ | 12  | 14 | 15  | 16+ | 19  | 24  | 27  | 29+ | 31  | 36+ |


  
**Answer the following questions.**

1. [1 marks] Enter the data for both the test and control groups into R and code the variables
appropriately.

<font color=RoyalBlue> Code explanation here... </font>

In [ ]:
QuestionEInput = ('
TestGroup RecurrenceTime Status
1 12 1
1 6 1
1 6 1
1 6 0
1 7 1
1 9 0
1 10 1
1 10 0
1 11 0
1 13 1
1 17 0
1 18 1
1 19 0
1 20 0
1 22 1
1 24 1
1 25 0
1 32 0
1 32 0
1 34 0
1 35 0
1 36 0
2 1 1
2 2 1
2 2 1
2 3 1
2 4 1
2 4 1
2 5 1
2 5 1
2 8 1
2 8 1
2 10 1
2 11 1
2 11 0
2 12 1
2 14 1
2 15 1
2 16 0
2 19 1
2 24 1
2 27 1
2 29 0
2 31 1
2 36 0
')

QuestionEData = read.table(textConnection(QuestionEInput), header = TRUE)
head(QuestionEData)

# Here I'm creating a duplicate data object in order to Mitigate any compatibility issues with previous code as I need to add Varibles into for a question
QuestionEDataControlSurvObject = QuestionEData

2. [3 marks] Produce graphs of the Kaplan Meier estimate of the survival functions, S(t).
What do these graphs suggest?

<font color=RoyalBlue> Code explanation here... </font>

In [ ]:
# Here I'm Creating the survival object of the data
SurvivalObject <- with(QuestionEData, Surv(RecurrenceTime, Status))
# Now I'm finding the surviver functions
KaplanMeierSurvivalFunction <- survfit(SurvivalObject~TestGroup, data=QuestionEData, conf.type = "plain") 
# Here I am showing the results
summary(KaplanMeierSurvivalFunction)

In [ ]:
# Now I am plotting the graph
plot(KaplanMeierSurvivalFunction, xlab="Time to Recurrence",ylab="Survival probability", yscale=100, col=c("RoyalBlue","Green"), main="Survival Distribution with control", conf.int = FALSE, mark.time = TRUE) 
# Here I am adding a legend
legend("topright", title="Group", c("Test", "Control"), fill=c("RoyalBlue", "Green"))

<font color=Green> 
Write some bullshit about the graphs...
</font>

3. [3 marks] Produce a plot of the cumulative hazard functions. What do these graphs
suggest?

<font color=RoyalBlue> Code explanation here... </font>

In [ ]:
# I'm starting by Transforming the Data from 2 -> 0 for analysis
QuestionEData$TestGroup [QuestionEData$TestGroup == 2] <- 0
# Now I'm Fining the Cox Proportional Hazards for the data
CoxProportionalHazards <- coxph(SurvivalObject ~ TestGroup,data=QuestionEData)
CoxProportionalHazards
# Here I'm finding the Hazard Function
HazardFunction<-survfit(CoxProportionalHazards)
# Here I am plotting the hazard function
ggsurvplot(HazardFunction, fun = "cumhaz",data=QuestionEData, palette= 'Green')


<font color=Green> 
Write some more bullshit about the graphs...
</font>

4. [3 marks] For the control group, what is the (K-M) estimated probability of survival beyond
12 weeks? Ignoring the small sample size, give approximate 95% confidence limits for the
true value.

<font color=RoyalBlue> Code explanation here... </font>

In [ ]:
# Now I'm finding the surviver functions
KaplanMeierSurvivalFunctionControl <- survfit(SurvivalObject~TestGroup, data=QuestionEData, conf.type = "plain") 
# Here I am showing the results
summary(KaplanMeierSurvivalFunctionControl)



<font color=Green> 
The Kaplan Meier estimated probability of survival beyond 12 weeks is 43.04% with an Upper confidence level of 63.5% and a Lower confidence level of 22.6%.
</font>

5. [3 marks] Carry out a Logrank test to determine if there is evidence of a significant difference
in survival functions. What do you conclude?

<font color=RoyalBlue> Code explanation here... </font>

In [ ]:
# Here I'm Performing a Logrank test
survdiff(SurvivalObject ~ TestGroup,data=QuestionEData)

<font color=Green> 
I conclude that stats sucks 
</font>

6. [3 marks] Fit a Cox regression model to this data using treatment as a single predictor and
estimate the hazard ratio for the test treatment together with a 95% confidence interval.
Interpret your result.

<font color=RoyalBlue> Code explanation here... </font>